In [5]:
import warnings

import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.layers import Embedding, LSTM, GRU, Dense, SpatialDropout1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('../data/Indonesian Sentiment Twitter Dataset Labeled.csv', sep="\t")
df.head()

In [7]:
df.replace(-1, 0, inplace=True)

In [8]:
# Assuming the CSV file has a column named 'text' that contains the text of each document
corpus = df['Tweet'].tolist()
sentiments = df['sentimen'].values

In [9]:
sentiment_encode = {-1 : 0, 0 : 1, 1 : 2}
y = df['sentimen'].map(sentiment_encode).values

In [10]:
# Convert sentiments to binary labels
binary_labels = np.array([1 if x == 1 else 0 for x in sentiments])

In [11]:
# Preprocess labels (convert categorical labels to numerical labels)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [13]:
# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
X_seq = tokenizer.texts_to_sequences(corpus)

# Pad sequences
max_length = max(len(seq) for seq in X_seq)
X_pad = pad_sequences(X_seq, maxlen=max_length)

In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)


In [15]:
# Define model architecture for LSTM
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length))
model_lstm.add(SpatialDropout1D(0.2))
model_lstm.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(1, activation='sigmoid'))

In [17]:
# Compile the model
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
# Train the model
epochs = 1
batch_size = 64
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history_lstm = model_lstm.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[early_stopping])

In [20]:
# Evaluate the model on test data
_, accuracy_lstm = model_lstm.evaluate(X_test, y_test)

In [21]:
# Define model architecture for GRU
model_gru = Sequential()
model_gru.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length))
model_gru.add(SpatialDropout1D(0.2))
model_gru.add(GRU(100, dropout=0.2, recurrent_dropout=0.2))
model_gru.add(Dense(1, activation='sigmoid'))

In [22]:
# Compile the model
model_gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
# Train the model
history_gru = model_gru.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[early_stopping])

In [24]:
# Evaluate the model on test data
_, accuracy_gru = model_gru.evaluate(X_test, y_test)

In [25]:
# Print accuracy of LSTM and GRU models
print(f"Accuracy of LSTM model: {accuracy_lstm:.4f}")
print(f"Accuracy of GRU model: {accuracy_gru:.4f}")